In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

## Setup

We're going to download the collected works of Nietzsche to use as our data for this class.

In [2]:
from pathlib import Path

if os.name == 'nt':
    PATH = str(Path.home()) + "\\data\\nietzsche\\"
else:
    PATH='data/nietzsche/'

In [3]:
get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(f'{PATH}nietzsche.txt').read()
print('corpus length:', len(text))

corpus length: 600901


In [4]:
text[:400]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself '

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 86


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [6]:
chars.insert(0, "\0")

''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

Map from chars to indices and back again

In [7]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

*idx* will be the data we use from now own - it simply converts all the characters to their index (based on the mapping above)

In [8]:
idx = [char_indices[c] for c in text]

idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [9]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

## Three char model

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [10]:
cs=3
c1_dat = [idx[i]   for i in range(0, len(idx)-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-cs, cs)]

Our inputs

In [11]:
x1 = np.stack(c1_dat)
x2 = np.stack(c2_dat)
x3 = np.stack(c3_dat)

Our output

In [12]:
y = np.stack(c4_dat)

The first 4 inputs and outputs

In [13]:
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [14]:
y[:4]

array([30, 29,  1, 40])

In [15]:
x1.shape, y.shape

((200300,), (200300,))

### Create and train model

Pick a size for our hidden state

In [16]:
n_hidden = 256

The number of latent factors to create (i.e. the size of the embedding matrix)

In [17]:
n_fac = 42

In [18]:
class Char3Model(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)

        # The 'green arrow' from our diagram - the layer operation from input to hidden
        self.l_in = nn.Linear(n_fac, n_hidden)

        # The 'orange arrow' from our diagram - the layer operation from hidden to hidden
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        
        # The 'blue arrow' from our diagram - the layer operation from hidden to output
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, c1, c2, c3):
        in1 = F.relu(self.l_in(self.e(c1.long())))
        in2 = F.relu(self.l_in(self.e(c2.long())))
        in3 = F.relu(self.l_in(self.e(c3.long())))
        
        h = V(torch.zeros(in1.size()).cuda())
        h = F.tanh(self.l_hidden(h+in1))
        h = F.tanh(self.l_hidden(h+in2))
        h = F.tanh(self.l_hidden(h+in3))
        
        return F.log_softmax(self.l_out(h))

In [19]:
md = ColumnarModelData.from_arrays('.', [-1], np.stack([x1,x2,x3], axis=1), y, bs=512)

In [20]:
m = Char3Model(vocab_size, n_fac).cuda()

In [21]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*(V(xs)))

In [22]:
opt = optim.Adam(m.parameters(), 1e-2)

In [23]:
%time fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       2.09643  3.44492]                                                                                           

Wall time: 3.18 s


In [24]:
set_lrs(opt, 0.001)

In [25]:
%time fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.84632  3.49825]                                                                                           

Wall time: 3.17 s


### Test model

In [26]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [27]:
get_next('y. ')

'T'

In [28]:
get_next('ppl')

'i'

In [29]:
get_next(' th')

'e'

In [30]:
get_next('and')

' '

## Our first RNN!

### Create inputs

This is the size of our unrolled RNN.

In [31]:
cs=8

For each of 0 through 7, create a list of every 8th character with that starting point. These will be the 8 inputs to out model.

In [32]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs)]

Then create a list of the next character in each of these series. This will be the labels for our model.

In [33]:
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs)]

In [34]:
xs = np.stack(c_in_dat, axis=0)

In [35]:
xs.shape

(600893, 8)

In [36]:
y = np.stack(c_out_dat)

So each column below is one series of 8 characters from the text.

In [37]:
xs[:cs,:cs]

array([[40, 42, 29, 30, 25, 27, 29,  1],
       [42, 29, 30, 25, 27, 29,  1,  1],
       [29, 30, 25, 27, 29,  1,  1,  1],
       [30, 25, 27, 29,  1,  1,  1, 43],
       [25, 27, 29,  1,  1,  1, 43, 45],
       [27, 29,  1,  1,  1, 43, 45, 40],
       [29,  1,  1,  1, 43, 45, 40, 40],
       [ 1,  1,  1, 43, 45, 40, 40, 39]])

...and this is the next character after each sequence.

In [38]:
y[:cs]

array([ 1,  1, 43, 45, 40, 40, 39, 43])

### Create and train model

In [39]:
val_idx = get_cv_idxs(len(idx)-cs-1)

In [40]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [41]:
class CharLoopModel(nn.Module):
    # This is an RNN!
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = F.relu(self.l_in(self.e(c.long())))
            h = F.tanh(self.l_hidden(h+inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [42]:
m = CharLoopModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [43]:
%time fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       2.00897  2.00615]                                                                                           

Wall time: 15 s


In [44]:
set_lrs(opt, 0.001)

In [45]:
%time fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.72085  1.71706]                                                                                           

Wall time: 14.9 s


In [46]:
class CharLoopConcatModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = torch.cat((h, self.e(c.long())), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [47]:
m = CharLoopConcatModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [48]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [49]:
%time fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.89629  1.85125]                                                                                           

Wall time: 21 s


In [50]:
set_lrs(opt, 1e-4)

In [51]:
%time fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.78157  1.77748]                                                                                           

Wall time: 15.6 s


### Test model

In [52]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [53]:
get_next('for thos')

'e'

In [54]:
get_next('part of ')

't'

In [55]:
get_next('queens a')

'n'

## RNN with pytorch

In [56]:
class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs).long())
        outp,h = self.rnn(inp, h)
        
        return F.log_softmax(self.l_out(outp[-1]), dim=-1)

In [57]:
m = CharRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [58]:
it = iter(md.trn_dl)
*xs,yt = next(it)

In [59]:
t = m.e(V(torch.stack(xs)).long())
t.size()

torch.Size([8, 512, 42])

In [60]:
ht = V(torch.zeros(1, 512,n_hidden))
outp, hn = m.rnn(t, ht)
outp.size(), hn.size()

(torch.Size([8, 512, 256]), torch.Size([1, 512, 256]))

In [61]:
t = m(*V(xs))
t.size()

torch.Size([512, 86])

In [62]:
%time fit(m, md, 4, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

[ 0.       1.86804  1.84806]                                                                                           
[ 1.       1.68372  1.66993]                                                                                           
[ 2.       1.59654  1.59468]                                                                                           
[ 3.       1.54615  1.5463 ]                                                                                           

Wall time: 59.7 s


In [63]:
set_lrs(opt, 1e-4)

In [64]:
%time fit(m, md, 2, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[ 0.       1.46698  1.50855]                                                                                           
[ 1.       1.46403  1.50334]                                                                                           

Wall time: 27.8 s


### Test model

In [65]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [66]:
get_next('for thos')

'e'

In [67]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [68]:
get_next_n('for thos', 40)

'for those the same the same the same the same th'

## Multi-output model

### Setup

Let's take non-overlapping sets of characters this time

In [69]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1, cs)]

Then create the exact same thing, offset by 1, as our labels

In [70]:
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

In [71]:
xs = np.stack(c_in_dat)
xs.shape

(75112, 8)

In [72]:
ys = np.stack(c_out_dat)
ys.shape

(75112, 8)

In [73]:
xs[:cs,:cs]

array([[40, 42, 29, 30, 25, 27, 29,  1],
       [ 1,  1, 43, 45, 40, 40, 39, 43],
       [33, 38, 31,  2, 73, 61, 54, 73],
       [ 2, 44, 71, 74, 73, 61,  2, 62],
       [72,  2, 54,  2, 76, 68, 66, 54],
       [67,  9,  9, 76, 61, 54, 73,  2],
       [73, 61, 58, 67, 24,  2, 33, 72],
       [ 2, 73, 61, 58, 71, 58,  2, 67]])

In [74]:
ys[:cs,:cs]

array([[42, 29, 30, 25, 27, 29,  1,  1],
       [ 1, 43, 45, 40, 40, 39, 43, 33],
       [38, 31,  2, 73, 61, 54, 73,  2],
       [44, 71, 74, 73, 61,  2, 62, 72],
       [ 2, 54,  2, 76, 68, 66, 54, 67],
       [ 9,  9, 76, 61, 54, 73,  2, 73],
       [61, 58, 67, 24,  2, 33, 72,  2],
       [73, 61, 58, 71, 58,  2, 67, 68]])

### Create and train model

In [75]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [76]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=512)

In [77]:
class CharSeqRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs).long())
        outp,h = self.rnn(inp, h)
        return F.log_softmax(self.l_out(outp), dim=-1)

In [78]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [79]:
it = iter(md.trn_dl)
*xst,yt = next(it)

In [80]:
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

In [81]:
%time fit(m, md, 4, opt, nll_loss_seq)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

[ 0.       2.5932   2.40234]                                                                                           
[ 1.       2.28775  2.19678]                                                                                           
[ 2.       2.13833  2.0808 ]                                                                                           
[ 3.       2.04324  2.00834]                                                                                           

Wall time: 6.99 s


In [82]:
set_lrs(opt, 1e-4)

In [83]:
%time fit(m, md, 1, opt, nll_loss_seq)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.99426  1.99409]                                                                                           

Wall time: 1.78 s


### Identity init!

In [84]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [85]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))


    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.cuda.FloatTensor of size 256x256 (GPU 0)]

In [86]:
%time fit(m, md, 4, opt, nll_loss_seq)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

[ 0.       2.36393  2.18764]                                                                                           
[ 1.       2.0864   2.01141]                                                                                           
[ 2.       1.97811  1.95759]                                                                                           
[ 3.       1.91952  1.91043]                                                                                           

Wall time: 7.06 s


In [87]:
set_lrs(opt, 1e-3)

In [88]:
%time fit(m, md, 4, opt, nll_loss_seq)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

[ 0.       1.82454  1.84374]                                                                                           
[ 1.       1.81572  1.83614]                                                                                           
[ 2.       1.80675  1.82975]                                                                                           
[ 3.       1.80301  1.8267 ]                                                                                           

Wall time: 7.07 s


## Stateful model

### Setup

In [89]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

from pathlib import Path

if os.name == 'nt':
    PATH = str(Path.home()) + "\\data\\nietzsche\\"
else:
    PATH='data/nietzsche/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'



    Only loading the 'en' tokenizer.



In [90]:
ls "{PATH}"

 Volume in Laufwerk C: hat keine Bezeichnung.
 Volumeseriennummer: 2494-B701

 Verzeichnis von C:\Users\Christian Klukas\data\nietzsche

10.01.2018  16:34    <DIR>          .
10.01.2018  16:34    <DIR>          ..
10.01.2018  17:53    <DIR>          models
09.01.2018  20:19           600.901 nietzsche.txt
10.01.2018  17:32    <DIR>          trn
10.01.2018  17:32    <DIR>          val
               1 Datei(en),        600.901 Bytes
               5 Verzeichnis(se), 339.774.873.600 Bytes frei


In [91]:
with open(os.path.join(PATH, 'nietzsche.txt')) as f:
    text = f.readlines()
    text_line_length = len(text)
    trn_index = int(text_line_length*.8)
    trn = text[:trn_index]
    tst = text[trn_index:]
    train_file_name = os.path.join(TRN, "trn.txt")
    val_file_name = os.path.join(VAL, "val.txt")
    if not os.path.exists(train_file_name) or not os.path.exists(val_file_name):
        print("Create train/test-split of input data...")
        with open(train_file_name,'w') as f2:
            f2.writelines(trn)
        with open(val_file_name,'w') as f3:
            f3.writelines(tst)

In [92]:
ls "{PATH}trn"

 Volume in Laufwerk C: hat keine Bezeichnung.
 Volumeseriennummer: 2494-B701

 Verzeichnis von C:\Users\Christian Klukas\data\nietzsche\trn

10.01.2018  17:32    <DIR>          .
10.01.2018  17:32    <DIR>          ..
10.01.2018  17:32           498.874 trn.txt
               1 Datei(en),        498.874 Bytes
               2 Verzeichnis(se), 339.774.861.312 Bytes frei


In [93]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=8; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(942, 56, 1, 482979)

### RNN

In [94]:
class CharSeqStatefulRnn(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [95]:
m = CharSeqStatefulRnn(md.nt, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [96]:
%time fit(m, md, 4, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

[ 0.       1.88546  1.85068]                                                                                           
[ 1.       1.70551  1.69914]                                                                                           
[ 2.       1.62698  1.64332]                                                                                           
[ 3.       1.57007  1.60167]                                                                                           

Wall time: 20.3 s


In [97]:
set_lrs(opt, 1e-4)

%time fit(m, md, 4, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

[ 0.       1.49375  1.55173]                                                                                           
[ 1.       1.49708  1.55108]                                                                                           
[ 2.       1.49081  1.54075]                                                                                           
[ 3.       1.48396  1.53946]                                                                                           

Wall time: 20.3 s


### RNN loop

In [98]:
# From the pytorch source

def RNNCell(input, hidden, w_ih, w_hh, b_ih, b_hh):
    return F.tanh(F.linear(input, w_ih, b_ih) + F.linear(hidden, w_hh, b_hh))

In [99]:
class CharSeqStatefulRnn2(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNNCell(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp = []
        o = self.h
        for c in cs: 
            o = self.rnn(self.e(c), o)
            outp.append(o)
        outp = self.l_out(torch.stack(outp))
        self.h = repackage_var(o)
        return F.log_softmax(outp, dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [100]:
m = CharSeqStatefulRnn2(md.nt, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [101]:
%time fit(m, md, 4, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

[ 0.       1.88552  1.86426]                                                                                           
[ 1.       1.71482  1.71686]                                                                                           
[ 2.       1.62181  1.64223]                                                                                           
[ 3.       1.57204  1.60364]                                                                                           

Wall time: 31.5 s


### GRU

In [102]:
class CharSeqStatefulGRU(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.GRU(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [103]:
# From the pytorch source code - for reference

def GRUCell(input, hidden, w_ih, w_hh, b_ih, b_hh):
    gi = F.linear(input, w_ih, b_ih)
    gh = F.linear(hidden, w_hh, b_hh)
    i_r, i_i, i_n = gi.chunk(3, 1)
    h_r, h_i, h_n = gh.chunk(3, 1)

    resetgate = F.sigmoid(i_r + h_r)
    inputgate = F.sigmoid(i_i + h_i)
    newgate = F.tanh(i_n + resetgate * h_n)
    return newgate + inputgate * (hidden - newgate)

In [104]:
m = CharSeqStatefulGRU(md.nt, n_fac, 512).cuda()

opt = optim.Adam(m.parameters(), 1e-3)

In [105]:
%time fit(m, md, 6, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=6), HTML(value='')))

[ 0.       1.77681  1.75665]                                                                                           
[ 1.       1.59509  1.59525]                                                                                           
[ 2.       1.49676  1.52843]                                                                                           
[ 3.       1.45389  1.50353]                                                                                           
[ 4.       1.40783  1.46925]                                                                                           
[ 5.       1.38489  1.46827]                                                                                           

Wall time: 30.9 s


In [106]:
set_lrs(opt, 1e-4)

In [107]:
%time fit(m, md, 3, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[ 0.       1.29528  1.42927]                                                                                           
[ 1.       1.29966  1.42714]                                                                                           
[ 2.       1.29963  1.42194]                                                                                           

Wall time: 15.5 s


### Putting it all together: LSTM

In [108]:
from fastai import sgdr

n_hidden=512

In [109]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [110]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [111]:
os.makedirs(f'{PATH}models', exist_ok=True)

In [112]:
%time fit(m, md, 2, lo.opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[ 0.       1.83494  1.75374]                                                                                           
[ 1.       1.71823  1.63742]                                                                                           

Wall time: 16.7 s


In [113]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
%time fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

[ 0.       1.55699  1.49722]                                                                                           
[ 1.       1.59983  1.5268 ]                                                                                           
[ 2.       1.48273  1.4421 ]                                                                                           
[ 3.       1.61351  1.53909]                                                                                           
[ 4.       1.54666  1.48285]                                                                                           
[ 5.       1.45367  1.42187]                                                                                           
[ 6.       1.39774  1.39074]                                                                                           
[ 7.       1.59122  1.51746]                                                                                           
[ 8.       1.56986  1.50525]            

In [114]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
%time fit(m, md, 2**6-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=63), HTML(value='')))

[ 0.       1.31344  1.35022]                                                                                           
[ 1.       1.31025  1.35   ]                                                                                           
[ 2.       1.31232  1.34354]                                                                                           
[ 3.       1.30945  1.34851]                                                                                           
[ 4.       1.30729  1.34486]                                                                                           
[ 5.       1.30472  1.33439]                                                                                           
[ 6.       1.29297  1.33721]                                                                                           
[ 7.       1.30077  1.33614]                                                                                           
[ 8.       1.29138  1.33495]            

### Test

In [115]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [116]:
get_next('for thos')

'e'

In [117]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [118]:
%time print(get_next_n('for thos', 400))

for those who reconclusion is emphasized indifference over to the hypocrisy and profound. i play the "done," crientime."23=probablyimpulse for these maif, as if one must getting out by natures. he would be faithin that he always power 'fornature.=--the belief. i deep towards enlightening and the solution.whoever strictly than indevelopment it some garmen)?193. at last opposed to "philosophical spirit more
Wall time: 678 ms
